In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from src.filter import GaussianOutlierFilter, ConditionFilter, NaNFilter
from src.extractor import LeidenRepresentativeSelector
from src.utils import UMAPVisualizer

import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("/workspace/01_data-extractor/data/TG_GATE_tables/merged_animal_data_final.csv", encoding='shift-jis')

In [3]:
nanfilter = NaNFilter(n=10)
nanfiltered_data = nanfilter.fit_transform(data)

Filtering completed 23868 -> 23636


In [16]:
condition_filter = ConditionFilter({"DOSE_LEVEL": "High", "SINGLE_REPEAT_TYPE": "Repeat"}, and_logic=True)
conditioned_data = condition_filter.fit_transform(nanfiltered_data)

Filtering completed 23636 -> 2702


In [5]:
gauss_filter = GaussianOutlierFilter(1,)
filtered_data = gauss_filter.fit_transform(conditioned_data, ["ALP(IU/L)", "ALT(IU/L)"])

Alert! ALP(IU/L) の正規性が否定されました (p=0.0010)。IQR法に切り替えます。
Alert! ALT(IU/L) の正規性が否定されました (p=0.0010)。IQR法に切り替えます。
Filtering completed 5777 -> 669


In [ ]:
umap = UMAPVisualizer()
reduced_data = umap.fit_transform(data, start_index=13, end_index=56)

In [ ]:
selector = LeidenRepresentativeSelector(n_neighbors=10, resolution=1.0, pca_threshold=0.9)
_, clustered_indices = selector.fit_transform(data, start_index=13, end_index=56, visualize=True)

In [15]:
data.iloc[clustered_indices]

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,COMPOUND_NAME,SINGLE_REPEAT_TYPE,SEX_TYPE,STRAIN_TYPE,ADMINISTRATION_ROUTE_TYPE,ANIMAL_AGE(week),SACRIFICE_PERIOD,DOSE,DOSE_UNIT,DOSE_LEVEL,TERMINAL_BW(g),LIVER(g),KIDNEY_TOTAL(g),KIDNEY_R(g),KIDNEY_L(g),ALP(IU/L),TC(mg/dL),TG(mg/dL),PL(mg/dL),TBIL(mg/dL),DBIL(mg/dL),GLC(mg/dL),BUN(mg/dL),CRE(mg/dL),Na(meq/L),K(meq/L),Cl(meq/L),Ca(mg/dL),IP(mg/dL),TP(g/dL),RALB(g/dL),A/G,AST(IU/L),ALT(IU/L),LDH(IU/L),GTP(IU/L),RBC(x10_4/ul),Hb(g/dL),Ht(%),MCV(fL),MCH(pg),MCHC(%),Ret(%),Plat(x10_4/uL),WBC(x10_2/uL),Neu(%),Eos(%),Bas(%),Mono(%),Lym(%),PT(s),APTT(s),Fbg(mg/dL),IMAGE_ID_Kidney,IMAGE_ID_Liver
15123,493,12,1,promethazine,Single,Male,Crj:CD(SD)IGS,Gavage,6,24 hr,60,mg/kg,Middle,228.3,9.820,1.980,0.990,0.990,1739.0,79.0,126.0,152.0,0.02,0.00,178.0,15.0,0.20,142.0,4.20,106.0,10.90,7.4,5.60,3.20,1.30,64.0,37.0,183.0,1.0,621.0,13.1,39.2,63.1,21.1,33.5,6.6,114.6,105.6,19.0,1.0,0.0,2.0,77.0,16.7,22.3,326.0,48465.0,48463.0
18297,570,6,5,nitrosodiethylamine,Single,Male,Crj:CD(SD)IGS,Gavage,6,6 hr,10,mg/kg,Low,238.7,11.104,2.118,1.041,1.077,968.0,81.0,79.0,147.0,0.14,0.09,197.0,15.0,0.40,140.0,3.90,103.0,10.60,8.2,5.50,3.30,1.50,63.0,32.0,139.0,2.0,611.0,13.1,38.2,62.6,21.5,34.4,6.4,132.5,111.8,14.0,1.0,0.0,2.0,82.0,12.4,19.6,216.0,52490.0,52365.0
12912,435,7,3,chlorpropamide,Single,Male,Crj:CD(SD)IGS,Gavage,6,9 hr,30,mg/kg,Low,195.4,7.886,1.746,0.865,0.881,1215.0,75.0,55.0,127.0,0.03,0.00,171.0,9.0,0.20,143.0,4.10,107.0,10.80,9.6,5.50,2.60,0.90,70.0,34.0,66.0,1.0,624.0,13.3,40.4,64.7,21.3,32.9,7.9,118.5,95.5,20.0,1.0,0.0,3.0,75.0,15.1,17.1,302.0,20707.0,20561.0
16495,525,14,3,triazolam,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,8 day,1000,mg/kg,High,239.9,12.273,2.254,1.130,1.124,1689.0,78.0,112.0,165.0,0.03,0.02,177.0,17.0,0.20,143.0,4.30,107.0,11.20,7.3,5.80,2.70,0.90,63.0,40.0,125.0,1.0,674.0,13.6,40.4,59.9,20.2,33.7,5.2,122.6,95.2,16.0,1.0,0.0,3.0,79.0,12.9,18.9,272.0,42532.0,42400.0
5979,251,12,5,thioridazine,Single,Male,Crj:CD(SD)IGS,Gavage,6,24 hr,30,mg/kg,Middle,222.5,11.762,2.117,1.062,1.055,710.0,87.0,91.0,163.0,0.42,0.04,191.0,18.0,0.50,139.0,4.60,105.0,11.10,9.7,5.80,3.80,1.90,62.0,38.0,70.0,1.0,637.0,13.6,41.1,64.5,21.4,33.1,8.7,105.8,116.0,15.0,1.0,0.0,6.0,78.0,12.1,16.2,227.0,9872.0,9713.0
19396,588,2,4,ethanol,Single,Male,Crj:CD(SD)IGS,Gavage,6,6 hr,0,mg/kg,Control,209.5,9.456,1.874,0.968,0.906,1011.0,72.0,64.0,138.0,0.43,0.05,183.0,11.0,0.50,139.0,4.80,104.0,10.80,10.0,5.50,3.80,2.20,58.0,27.0,56.0,2.0,615.0,13.4,40.2,65.3,21.8,33.3,9.6,121.2,73.7,16.0,1.0,0.0,8.0,75.0,12.3,18.4,214.0,47906.0,47671.0
12592,430,7,3,caffeine,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,15 day,10,mg/kg,Low,309.7,14.320,2.500,1.280,1.220,1361.0,74.0,133.0,150.0,0.00,0.00,182.0,21.0,0.20,143.0,4.10,106.0,10.80,6.2,5.90,3.20,1.20,67.0,37.0,151.0,2.0,702.0,14.3,41.0,58.4,20.3,34.8,4.4,116.1,104.7,13.0,1.0,0.0,1.0,84.0,14.9,20.9,322.0,36716.0,36715.0
17902,553,7,5,benziodarone,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,15 day,30,mg/kg,Low,332.6,16.853,2.625,1.353,1.272,745.0,77.0,144.0,162.0,0.29,0.00,190.0,18.0,0.60,140.0,4.40,104.0,10.40,7.0,6.10,3.90,1.80,59.0,24.0,84.0,2.0,693.0,14.4,42.3,61.0,20.7,33.9,2.9,121.1,115.0,13.0,1.0,0.0,7.0,79.0,11.8,17.1,222.0,45133.0,44970.0
2492,151,3,3,nitrofurantoin,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,15 day,0,mg/kg,Control,341.8,15.038,2.620,1.275,1.345,752.0,76.0,151.0,161.0,0.16,0.11,230.0,18.0,0.40,140.0,3.60,102.0,10.50,6.8,6.20,3.60,1.40,62.0,41.0,166.0,1.0,725.0,15.0,42.8,59.0,20.7,35.1,4.4,121.0,118.2,17.0,1.0,0.0,2.0,78.0,11.9,19.0,234.0,30200.0,30020.0
16937,534,6,5,terbinafine,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,8 day,75,mg/kg,Low,267.3,12.045,2.253,1.076,1.177,940.0,81.0,63.0,150.0,0.12,0.07,183.0,14.0,0.40,142.0,4.10,104.0,10.60,6.7,5.60,3.30,1.40,79.0,36.0,160.0,1.0,621.0,13.7,39.3,63.4,22.1,34.9,5.4,118.6,112.8,16.0,1.0,1.0,2.0,79.0,12.0,18.8,216.0,44377.0,44168.0
